In [11]:
import json
import psycopg2
import unicodedata
from psycopg2.extras import execute_values

In [12]:
# Load the JSON file
with open('lnkdn.json', 'r') as file:
    combined_records = json.load(file)

# Display the loaded data
combined_records

[{'message_id': 'messg_0',
  'conversation_id': 'conv_0',
  'platform': 'LinkedIn',
  'connection_name': 'Namruth Thimmapuram',
  'message_sender': 'Vikas Reddy Venkannagari',
  'message_date': '2024-11-04 20:28:00',
  'message_text': "Hi Namruth Goud,\n\nIt's been some time since we connected. We both studied at University of Maryland Baltimore County. How have you been?",
  'connection_title': "Master's in Data Science @UMBC"},
 {'message_id': 'messg_0',
  'conversation_id': 'conv_1',
  'platform': 'LinkedIn',
  'connection_name': 'Namruth Thimmapuram',
  'message_sender': 'Namruth Thimmapuram',
  'message_date': '2024-11-10 23:25:00',
  'message_text': 'Hi, Vikas Reddy',
  'connection_title': "Master's in Data Science @UMBC"},
 {'message_id': 'messg_0',
  'conversation_id': 'conv_2',
  'platform': 'LinkedIn',
  'connection_name': 'Namruth Thimmapuram',
  'message_sender': 'Namruth Thimmapuram',
  'message_date': '2024-11-10 23:25:00',
  'message_text': "I'm good",
  'connection_titl

In [13]:
# Connect to the PostgreSQL database
connection = psycopg2.connect(
    dbname="messages",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)

# Create a cursor object
cursor = connection.cursor()

# Print connection status
print("Connected to the database")

Connected to the database


### filter out past messages from being inserted into the postgres

In [ ]:
# Create a set to track unique (message_id, conversation_id) pairs
unique_pairs = set()

# Filter out duplicates
filtered_records = []
for record in combined_records:
    pair = (record['message_id'], record['conversation_id'])
    cursor.execute("""
    SELECT 1 FROM linkedin_messages 
    WHERE message_id = %s AND conversation_id = %s
    """, pair)
    if not cursor.fetchone():
        unique_pairs.add(pair)
        filtered_records.append(record)

print(f"Filtered records count: {len(filtered_records)}")

comb_records = combined_records
combined_records = filtered_records

Filtered records count: 0


In [14]:
# Preprocess text
def preprocess_text(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')

# Process the "message_text" field in combined_records
for record in combined_records:
    record["message_text"] = preprocess_text(record["message_text"])


### Code to sent the messages into the table

In [ ]:

# 3. Create a new table
cursor.execute("""
CREATE TABLE IF NOT EXISTS linkedin_messages (
  message_id       TEXT,
  conversation_id  TEXT,
  platform         TEXT,
  connection_name  TEXT,
  message_sender   TEXT,
  message_date     TIMESTAMP,
  message_text     TEXT,
  connection_title TEXT,
  PRIMARY KEY (message_id, conversation_id)
);
""")

# 4. Prepare bulk‐insert
sql = """
INSERT INTO linkedin_messages
  (message_id, conversation_id, platform,
   connection_name, message_sender, message_date,
   message_text, connection_title)
VALUES %s
ON CONFLICT DO NOTHING;
"""

values = [
    (
      rec["message_id"],
      rec["conversation_id"],
      rec["platform"],
      rec["connection_name"],
      rec["message_sender"],
      rec["message_date"],        # ISO format → psycopg2 parses to TIMESTAMP
      rec["message_text"],
      rec.get("connection_title")
    )
    for rec in combined_records
]

# 5. Execute and commit
execute_values(cursor, sql, values)
connection.commit()

In [36]:
# 6. Cleanup
cursor.close()
connection.close()

print(f"Inserted {len(values)} messages (duplicates skipped).")

Inserted 1381 messages (duplicates skipped).


### Code to create a 2nd table linkedin_summary, to store the summaries of conversations. 

In [57]:
# Create a new table for the summarized data
cursor.execute("""
CREATE TABLE IF NOT EXISTS linkedin_summary (
    connection_name TEXT,
    message_id      TEXT PRIMARY KEY,
    conversation_id TEXT,
    message_sender  TEXT, 
    message_date    TIMESTAMP
               
);
""")



In [ ]:
message_ids = set()
for index in combined_records:
    message_ids.add(index["message_id"])

print("the total number of records are: ", len(message_ids))

s_records = []

# going through all the records and finding the most recent interaction.
for i in message_ids:
    maxi_convid = -1
    for index in combined_records:
        if i == index["message_id"]:
            conv_id = int(index['conversation_id'].replace("conv_", ""))

            if conv_id > maxi_convid:
                maxi_convid = conv_id
                conversation_id = index["conversation_id"]
                message_sender = index["message_sender"]
                connection_name = index["connection_name"]
                message_date = index["message_date"]
            
    print({"connection_name": connection_name, 
        "message_id": i,
        "conversation_id": maxi_convid,
        "message_sender": message_sender})
    s_records.append({"connection_name" : connection_name, 
                    "message_id" : i,
                    "conversation_id" : conversation_id,
                    "message_sender" : message_sender,
                    "message_date" : message_date})



{'connection_name': 'Chun-Yen Lee', 'message_id': 'messg_67', 'conversation_id': 0, 'message_sender': 'Vikas Reddy Venkannagari'}
{'connection_name': 'Haixu Leng', 'message_id': 'messg_1', 'conversation_id': 0, 'message_sender': 'Vikas Reddy Venkannagari'}
{'connection_name': 'Shawn Kerrigan', 'message_id': 'messg_213', 'conversation_id': 0, 'message_sender': 'Vikas Reddy Venkannagari'}
{'connection_name': 'Sathvika Anugu', 'message_id': 'messg_95', 'conversation_id': 0, 'message_sender': 'Vikas Reddy Venkannagari'}
{'connection_name': 'Anurag Ganguli', 'message_id': 'messg_212', 'conversation_id': 0, 'message_sender': 'Vikas Reddy Venkannagari'}
{'connection_name': 'Umme Kulsum', 'message_id': 'messg_41', 'conversation_id': 4, 'message_sender': 'Umme Kulsum'}
{'connection_name': 'Christopher Rankin', 'message_id': 'messg_59', 'conversation_id': 0, 'message_sender': 'Vikas Reddy Venkannagari'}
{'connection_name': 'Phanindra Chowdary B', 'message_id': 'messg_171', 'conversation_id': 1, 

In [ ]:
summary_sql = """
INSERT INTO linkedin_summary
  (connection_name, message_id, conversation_id, message_sender, message_date)
VALUES %s
ON CONFLICT DO NOTHING;
"""

# Prepare the values for insertion
values = [
  (
    rec["connection_name"],
    rec["message_id"],
    rec["conversation_id"],
    rec["message_sender"],
    rec["message_date"]
  )
  for rec in s_records
]

# Execute and commit
execute_values(cursor, summary_sql, values)
connection.commit()

# 6. Cleanup
# cursor.close()
# connection.close()


## Part 2 : Generating Summary

### generate the summaries for all the conversations

In [24]:
def get_conversation_by_message_id(conn, message_id):
    """Retrieve all messages for a given message_id and format as a conversation."""
    try:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT message_sender, message_date, message_text, connection_title
            FROM linkedin_messages
            WHERE message_id = %s
            ORDER BY message_date ASC
        """, (message_id,))
        rows = cursor.fetchall()
        cursor.close()
        
        # Format conversation as a string
        conversation = ""
        for row in rows:
            sender, date, text, title = row
            conversation += f"{sender} ({date}): {text}\n"
        return conversation.strip(), title
    except Exception as e:
        print(f"Error fetching conversation for message_id {message_id}: {e}")
        return ""

In [44]:
from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI

# Load environment variables from .env file
load_dotenv()

# Get the SUMMARIES_OPENAI variable
api_key_openai = os.getenv("SUMMARIES_OPENAI")

# Initialize OpenAI model via LangChain
llm = ChatOpenAI(
    openai_api_key=api_key_openai,
    temperature=0.3,
    max_tokens=1024,
    model="gpt-3.5-turbo",
    verbose=True,
)

message_ids = set()
for index in combined_records:
    message_ids.add(index["message_id"])

print("the total number of records are: ", len(message_ids))


the total number of records are:  223


In [45]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_ollama import ChatOllama


# Define prompt template for summarization
prompt_template = PromptTemplate(
    input_variables=["conversation"],
    template="""You are a helpful assistant that summarizes LinkedIn chat conversations.

        Summarize the conversation below in 3–5 sentences. Focus on:
        - Who I messaged and their role/company
        - My intent (e.g., asking for a referral, meeting, or recruiter intro)
        - Their response and any follow-up actions
        - Whether a resume, project, or portfolio was shared
        - Outcome or current status (e.g., they agreed, ignored, redirected)

        The summary should help me remember this conversation later, without rereading it.
        \n\n{conversation}"""
)

# Create LLM chain for summarization
summary_chain = LLMChain(llm=llm, prompt=prompt_template)

def generate_summary(conversation):
    """Generate a summary for the given conversation using LangChain and Ollama."""
    try:
        summary = summary_chain.run(conversation=conversation)
        return summary.strip()
    except Exception as e:
        print(f"Error generating summary: {e}")
        return "Summary generation failed."



# Define a new prompt template for extracting the company name in JSON format
json_prompt_template = PromptTemplate(
    input_variables=["conversation", "title"],
    template="""You are an assistant that extracts the company name from LinkedIn chat conversations.

        Based on the conversation and title provided below, identify the company name. If the company cannot be determined, return "None".

        Output the result in JSON format with the following structure:
        {{
            "company_name": "<company_name or None>"
        }}

        Conversation: {conversation}
        Title: {title}
    """
)

# Create a new LLM chain for extracting the company name
json_chain = LLMChain(llm=llm, prompt=json_prompt_template)

def extract_company_name(conversation, title):
    """Extract the company name from the conversation and title using LangChain."""
    try:
        result = json_chain.run(conversation=conversation, title=title)
        return result.strip()
    except Exception as e:
        print(f"Error extracting company name: {e}")
        return '{"company_name": "None"}'

In [46]:
summaries = []
from tqdm import tqdm  

for message_id in tqdm(message_ids):
    # Retrieve rows with the same message_id from the database
    
    conversation, title = get_conversation_by_message_id(connection, message_id)
    
    # Generate summary
    summary = generate_summary(conversation)

    company = extract_company_name(conversation, title)
    clean = company.replace('```json\n', '').replace('\n```', '')
    data = json.loads(clean)

    # Store summary with metadata
    summaries.append({
        "message_id": message_id,
        "conversation": conversation,
        "summary": summary,
        "company": data["company_name"]
    })  

100%|██████████| 223/223 [05:00<00:00,  1.35s/it]


In [47]:
summaries

[{'message_id': 'messg_82',
  'conversation': 'Vikas Reddy Venkannagari (2025-04-15 19:58:00): Hey Anusha, Im Vikas, a current grad student at UMBC, applying for Data Analyst role at Concentrix. Id love your takewould you be open to a referral? I bring over 1.5 years of relevant experience and am happy to share my resume. Thanks!\nAnusha Pugazhendhi (2025-04-15 19:59:00): Hi Vikas, yes sure. Please give me your full name, address and resume\nAnusha Pugazhendhi (2025-04-15 19:59:00): Also send the job link here \nVikas Reddy Venkannagari (2025-04-18 14:09:00): Sorry for the late reply Anusha, and thanks for offering me a referral. The role that I was interested was closed, and all the other roles seems not be aligned for new grads, I will let you know if I find a good fit.',
  'summary': "I messaged Anusha Pugazhendhi, who works at Concentrix, asking for a referral for a Data Analyst role. Anusha agreed to provide a referral and requested my full name, address, resume, and the job link.

In [49]:
# Add new columns to the linkedin_summary table
cursor.execute("""
ALTER TABLE linkedin_summary
ADD COLUMN IF NOT EXISTS chat_summary TEXT,
ADD COLUMN IF NOT EXISTS company TEXT;
""")

# Commit the changes
connection.commit()

print("Columns 'chat_summary' and 'company' added successfully.")

Columns 'chat_summary' and 'company' added successfully.


In [ ]:
# Update the linkedin_summary table with summary and company for each message_id
update_sql = """
UPDATE linkedin_summary
SET chat_summary = %s, company = %s
WHERE message_id = %s;
"""
 
for summary_record in summaries:
    try:
        cursor.execute(update_sql, (summary_record["summary"], summary_record["company"], summary_record["message_id"]))
    except Exception as e:
        print(f"Error updating message_id {summary_record['message_id']}: {e}")

# Commit the changes
connection.commit()

print("Summary and company information updated successfully.")

Summary and company information updated successfully.


In [51]:
print(data)

{'company_name': 'Glimpse'}
